In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.metrics import r2_score
import utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.tree import DecisionTreeRegressor, plot_tree
import matplotlib as plt

In [ ]:
airbnb = pd.read_csv(
    "../Data/Airbnb_data_categorial.csv",
)

#prediction
airbnb['price'] = np.log(airbnb.price+1)

# Features und Zielvariable trennen
X = airbnb.drop("price", axis=1)  # Alle Spalten außer "price" sind Features
y = airbnb["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Aufteilung des Trainingsdatensatzes in Trainings- und Validierungsdatensatz (80% Trainingsdaten, 20% Validierungsdaten)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# 5. Datennormalisierung
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [ ]:
dt = DecisionTreeRegressor(min_samples_leaf=200)
dt.fit(X_train, y_train)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plot_tree(dt, filled=True, ax=ax, fontsize=12, feature_names=X_train.columns);

### Random Forest

In [ ]:
rf = RandomForestRegressor(n_estimators=10, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
utils.print_scores(m=rf, X_train=X_train, X_valid=X_val)

### Feature Importance

In [ ]:
from sklearn.inspection import permutation_importance
import seaborn as sns

In [ ]:
permutation_importance(rf, X_train, y_train, n_repeats=1,n_jobs=-1)

In [ ]:
# Create a new dataframe for easy plotting
fi_df = pd.DataFrame(
    data={'Feature': X_train.columns, 'Feature importance': rf.feature_importances_}, 
    columns = ['Feature', 'Feature importance']
)
fi_df.sort_values('Feature importance', inplace=True, ascending=False)
sns.barplot(data=fi_df, x='Feature importance', y='Feature')
plt.xscale('log')

### Feature importance: Partial dependence

In [ ]:
from sklearn.inspection import plot_partial_dependence
order = np.argsort(rf.feature_importances_)[::-1]
fig, ax = plt.subplots(figsize=(18, 3))
plot_partial_dependence(rf, X_val[::1000], order[1:8], feature_names=X_train.columns, grid_resolution=5, n_jobs=-1, n_cols=7, ax=ax)
plt.tight_layout()